1. Read in the provided data file using pandas and convert missing data to NA values.

In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

data = pd.read_csv("~/work/exercises/final-exercise-hennaylimaa/data/2315676.txt", na_values=-9999, skiprows=[1], delim_whitespace=True)

print("Length of data before remove of null values:", data.shape[0])

Length of data before remove of null values: 41065


2. Fill in the missing values in the `TAVG` column with your estimates of the average daily temperature as noted above in the data section. You can then drop any average daily temperatures that are still missing.

In [139]:
data["AVG"] = (data['TMAX'] + data['TMIN']) / 2 #data[['TMAX', 'TMIN']].mean(axis=1)
data.loc[data['TAVG'].isna() == True, 'TAVG'] = data['AVG']
data = data.dropna(subset=['TAVG'])

print("Length of data after remove of null values:", data.shape[0])
data.head()

Length of data after remove of null values: 40882


,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN,AVG
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,-17.5,2.0,-37.0,-17.5
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,-10.0,6.0,-26.0,-10.0
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,-10.0,7.0,-27.0,-10.0
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,-15.0,-3.0,-27.0,-15.0
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,-16.0,4.0,-36.0,-16.0


3. Define and use a function to convert temperatures in Fahrenheit to Celsius.

In [140]:
"""Function to convert Fahrenheit temperature into Celsius.

    Parameters:
        temp_fahrenheit: int | float
            Input temperature in Fahrenheit (should be a number)

    Returns:
        Temperature in Celsius (float)
    
    Author:
        Henna Ylimaa - 8.3.2022
"""
# function that converts fahrenheits to celsius
# one value in fahrenheit is given to the funtion
def fahr_to_celsius(temp_fahrenheit):
    
    # converts temp_fahrenheit to celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8
    
    # converted_temp that have the celsius value is returned
    return converted_temp

data["TAVG_C"] = data["TAVG"].apply(fahr_to_celsius)
data["TMAX_C"] = data["TMAX"].apply(fahr_to_celsius)
data["TMIN_C"] = data["TMIN"].apply(fahr_to_celsius)
data["AVG_C"] = data["AVG"].apply(fahr_to_celsius)
data.head()

,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN,AVG,TAVG_C,TMAX_C,TMIN_C,AVG_C
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,-17.5,2.0,-37.0,-17.5,-27.500000,-16.666667,-38.333333,-27.500000
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,-10.0,6.0,-26.0,-10.0,-23.333333,-14.444444,-32.222222,-23.333333
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,-10.0,7.0,-27.0,-10.0,-23.333333,-13.888889,-32.777778,-23.333333
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,-15.0,-3.0,-27.0,-15.0,-26.111111,-19.444444,-32.777778,-26.111111
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,-16.0,4.0,-36.0,-16.0,-26.666667,-15.555556,-37.777778,-26.666667


4. Calculate seasonal average temperatures for each season in every year (e.g., Winter 1909, Spring 1909, Summer 1909, ...)

In [141]:
data.loc[:,'DATE'] = pd.to_datetime(data['DATE'], format='%Y%m%d')
# season column
def season_from_dt(date):
    if date.month in (12, 1, 2):
        return 1
    elif date.month in (3, 4, 5):
        return 2
    elif date.month in (6, 7, 8):
        return 3
    elif date.month in (9, 10, 11):
        return 4
data['season'] = data['DATE'].apply(season_from_dt)
data['year'] = pd.DatetimeIndex(data['DATE']).year
seasonal_averages = data.groupby(['year', 'season'], as_index=False)['TAVG_C'].mean()
seasonal_averages.head(50)

,year,season,TAVG_C
0,1908,1,-12.330779
1,1908,2,-2.346743
2,1908,3,11.425926
3,1908,4,-1.859217
4,1909,1,-11.580933
5,1909,2,-5.487654
6,1909,3,11.814614
7,1909,4,-0.155827
8,1910,1,-9.471767
9,1910,2,-0.122549


5. Calculate seasonal average temperatures for the reference period 1951-1980 (e.g., 4 values in total, one for each season)

In [ ]:
# subsetti vuosista ja näistä average

6. Calculate seasonal temperature anomalies for each year

In [ ]:
# suhteutetaan referenssidataan?

7. Plot the data as shown in the example above